In [6]:
import random
import time
import logging
import unittest
from enum import Enum, auto
from typing import List, Tuple, Dict

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(message)s')

class TimeOfDay(Enum):
    MORNING = auto()
    DAY = auto()
    EVENING = auto()
    NIGHT = auto()

class EcosystemMeta(type):
    """Метакласс для управления классами экосистемы"""
    _registry: Dict[str, Dict[str, type]] = {
        'plants': {},
        'animals': {}
    }

    def __new__(cls, name, bases, namespace, **kwargs):
        """Создание класса с инжекцией поведения"""
        new_class = super().__new__(cls, name, bases, namespace)
        
        # Регистрация классов растений
        if 'Plant' in [b.__name__ for b in bases] and name != 'Plant':
            cls._registry['plants'][name] = new_class
            cls._inject_plant_methods(new_class, namespace.get('behavior', {}))
        
        # Регистрация классов животных
        elif 'Animal' in [b.__name__ for b in bases] and name != 'Animal':
            cls._registry['animals'][name] = new_class
            cls._inject_animal_methods(new_class, namespace.get('behavior', {}))
        
        return new_class

    @classmethod
    def _inject_plant_methods(cls, target_class: type, behavior: dict):
        """Инжект методов для растений"""
        def spread(self):
            """Распространение растения с учетом поведения"""
            if not self.is_active(self.world.time_of_day):
                return

            neighbors = self.world.get_neighbors(self.x, self.y)
            for nx, ny in random.sample(neighbors, k=len(neighbors)):
                target = self.world.grid[ny][nx]

                # Попытка размножения
                if target is None and random.random() < behavior.get('spread_chance', 0.3):
                    if self.world.add_entity(self.__class__(), nx, ny):
                        self.energy -= 10

                # Конкурентная борьба
                elif (isinstance(target, Plant) and 
                      self.is_active(self.world.time_of_day) and 
                      not target.is_active(self.world.time_of_day) and 
                      random.random() < behavior.get('competitive_chance', 0.5)):
                    target.die()

        # Динамическое добавление методов
        target_class.spread = spread
        
        # Добавление проверки активности
        def is_active(self, time_of_day: TimeOfDay):
            self.active = time_of_day in behavior.get('active_times', [])
            return self.active
        target_class.is_active = is_active
        
        return target_class

    @classmethod
    def _inject_animal_methods(cls, target_class: type, behavior: dict):
        """Инжект методов для животных"""
        def move(self):
            """Логика движения с учетом поведения"""
            if self.is_sleeping():
                return

            # Расчет скорости
            speed = behavior.get('base_speed', 1.0)
            if self.hunger > behavior.get('hunger_threshold', 50):
                speed *= behavior.get('hungry_speed_mod', 0.8)

            # Выбор доступных клеток
            possible_moves = [
                (x, y) for x, y in self.world.get_neighbors(self.x, self.y)
                if self.world.grid[y][x] is None
            ]
            
            if possible_moves and random.random() < speed:
                nx, ny = random.choice(possible_moves)
                self.world.move_entity(self, nx, ny)

        # Динамическое добавление методов
        target_class.move = move
        
        def eat(self):
            """Логика питания"""
            if self.is_sleeping() or self.hunger < 20:
                return

            # Поиск подходящей пищи
            for food_type in behavior.get('food_types', []):
                for x, y in self.world.get_neighbors(self.x, self.y):
                    target = self.world.grid[y][x]
                    if isinstance(target, food_type):
                        nutrition = behavior.get('nutrition', 30)
                        self.hunger = max(0, self.hunger - nutrition)
                        self.energy += behavior.get('energy_gain', 20)
                        target.die()
                        return

        target_class.eat = eat
        
        # Добавление проверки сна
        def is_sleeping(self):
            return self.world.time_of_day in behavior.get('sleep_times', [])
        target_class.is_sleeping = is_sleeping
        
        return target_class

class Plant(metaclass=EcosystemMeta):
    """Базовый класс растений"""
    def __init__(self):
        self.x = 0
        self.y = 0
        self.world = None
        self.energy = 100
        self.active = False

    def die(self):
        """Уничтожение растения"""
        self.world.remove_entity(self)

class Animal(metaclass=EcosystemMeta):
    """Базовый класс животных"""
    def __init__(self):
        self.x = 0
        self.y = 0
        self.world = None
        self.energy = 100
        self.hunger = 0

    def reproduce(self):
        """Логика размножения"""
        if self.energy < 80 or random.random() > 0.1:
            return

        for x, y in self.world.get_neighbors(self.x, self.y):
            if self.world.grid[y][x] is None:
                newborn = self.__class__()
                if self.world.add_entity(newborn, x, y):
                    self.energy -= 40
                    return

    def die(self):
        """Уничтожение животного"""
        self.world.remove_entity(self)

# Реализации растений
class Lumiere(Plant):
    behavior = {
        'active_times': [TimeOfDay.DAY],
        'spread_chance': 0.4,
        'competitive_chance': 0.6
    }

class Obscurite(Plant):
    behavior = {
        'active_times': [TimeOfDay.NIGHT],
        'spread_chance': 0.5,
        'competitive_chance': 0.7
    }

class Demi(Plant):
    behavior = {
        'active_times': [TimeOfDay.MORNING, TimeOfDay.EVENING],
        'spread_chance': 0.3,
        'competitive_chance': 0.5
    }

# Реализации животных
class Pauvre(Animal):
    behavior = {
        'food_types': [Lumiere],
        'sleep_times': [TimeOfDay.NIGHT],
        'base_speed': 1.0,
        'hungry_speed_mod': 0.7,
        'hunger_threshold': 60,
        'nutrition': 40,
        'energy_gain': 25
    }

class Malheureux(Animal):
    behavior = {
        'food_types': [Demi, Obscurite, Pauvre],
        'sleep_times': [TimeOfDay.DAY],
        'base_speed': 0.9,
        'hungry_speed_mod': 0.6,
        'nutrition': 50,
        'energy_gain': 30
    }

class World:
    """Класс мира для симуляции"""
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
        self.grid = [[None for _ in range(width)] for _ in range(height)]
        self.time_of_day = TimeOfDay.MORNING
        self.entities = []
        self.tick = 0

    def add_entity(self, entity, x: int, y: int) -> bool:
        """Добавление сущности в мир"""
        if not (0 <= x < self.width and 0 <= y < self.height):
            return False

        if self.grid[y][x] is not None:
            return False

        entity.x = x
        entity.y = y
        entity.world = self
        self.grid[y][x] = entity
        self.entities.append(entity)
        return True

    def remove_entity(self, entity):
        """Удаление сущности из мира"""
        if entity in self.entities:
            self.entities.remove(entity)
        if 0 <= entity.x < self.width and 0 <= entity.y < self.height:
            self.grid[entity.y][entity.x] = None

    def move_entity(self, entity, new_x: int, new_y: int):
        """Перемещение сущности"""
        self.grid[entity.y][entity.x] = None
        entity.x = new_x
        entity.y = new_y
        self.grid[new_y][new_x] = entity

    def get_neighbors(self, x: int, y: int) -> List[Tuple[int, int]]:
        """Получение соседних клеток"""
        return [
            (x + dx, y + dy)
            for dx in (-1, 0, 1)
            for dy in (-1, 0, 1)
            if (dx != 0 or dy != 0) 
            and 0 <= x + dx < self.width 
            and 0 <= y + dy < self.height
        ]

    def update_time(self):
        """Обновление времени суток"""
        self.tick += 1
        if self.tick % 6 == 0:
            self.time_of_day = {
                TimeOfDay.MORNING: TimeOfDay.DAY,
                TimeOfDay.DAY: TimeOfDay.EVENING,
                TimeOfDay.EVENING: TimeOfDay.NIGHT,
                TimeOfDay.NIGHT: TimeOfDay.MORNING
            }[self.time_of_day]

    def step(self):
        """Шаг симуляции"""
        self.update_time()
        
        # Обработка растений
        for entity in self.entities.copy():
            if isinstance(entity, Plant):
                entity.spread()
        
        # Обработка животных
        for entity in self.entities.copy():
            if isinstance(entity, Animal):
                entity.move()
                entity.eat()
                entity.reproduce()
                
                # Обновление состояния
                entity.hunger += 5
                if entity.hunger > 100:
                    entity.die()
                elif entity.energy <= 0:
                    entity.die()

    def display(self):
        """Визуализация мира"""
        time_names = {
            TimeOfDay.MORNING: "Утро",
            TimeOfDay.DAY: "День",
            TimeOfDay.EVENING: "Вечер",
            TimeOfDay.NIGHT: "Ночь"
        }
        print(f"\nВремя: {time_names[self.time_of_day]} (Тик: {self.tick})")
        print(f"Сущностей: {len(self.entities)}")
        
        for row in self.grid:
            print(' '.join(
                'L' if isinstance(c, Lumiere) else
                'O' if isinstance(c, Obscurite) else
                'D' if isinstance(c, Demi) else
                'P' if isinstance(c, Pauvre) else
                'M' if isinstance(c, Malheureux) else '.'
                for c in row
            ))

class TestEcosystem(unittest.TestCase):
    """Тесты экосистемы"""
    def setUp(self):
        self.world = World(10, 10)
        self.world.time_of_day = TimeOfDay.DAY

    def test_plant_registration(self):
        self.assertIn('Lumiere', EcosystemMeta._registry['plants'])
        self.assertIn('Obscurite', EcosystemMeta._registry['plants'])
        self.assertIn('Demi', EcosystemMeta._registry['plants'])

    def test_animal_registration(self):
        self.assertIn('Pauvre', EcosystemMeta._registry['animals'])
        self.assertIn('Malheureux', EcosystemMeta._registry['animals'])

    def test_plant_spread(self):
        plant = Lumiere()
        self.world.add_entity(plant, 5, 5)
        plant.spread()
        self.assertGreaterEqual(len(self.world.entities), 1)

def main():
    """Запуск симуляции"""
    # Тестирование
    suite = unittest.TestLoader().loadTestsFromTestCase(TestEcosystem)
    runner = unittest.TextTestRunner()
    result = runner.run(suite)
    
    if result.wasSuccessful():
        # Демонстрация
        world = World(15, 15)
        
        # Инициализация
        for _ in range(20):
            x, y = random.randint(0, 14), random.randint(0, 14)
            world.add_entity(random.choice([Lumiere, Obscurite, Demi])(), x, y)
        
        for _ in range(5):
            x, y = random.randint(0, 14), random.randint(0, 14)
            world.add_entity(Pauvre(), x, y)
        
        for _ in range(3):
            x, y = random.randint(0, 14), random.randint(0, 14)
            world.add_entity(Malheureux(), x, y)
        
        # Запуск цикла
        for _ in range(10):
            world.step()
            world.display()
            time.sleep(1)

if __name__ == "__main__":
    main()

...
----------------------------------------------------------------------
Ran 3 tests in 0.002s

OK



Время: Утро (Тик: 1)
Сущностей: 35
. . L . M . . . . . . . . . .
. . . . D . . . . . . . . . .
O . . D . . . . . O . . . . .
. . P . . . . L . . . . . . .
. . . P . . . . . O O . . . .
. . . . . M . . . . L . . . .
. . . . . . . L . . . . . . .
. . . L . . . . D . L . . . .
. O . . . . . . D . . . . . .
. . . . . . . . D O . . . . .
. . . . . . . . . . . . . D .
. . . . . . . . . . . . . D D
. . . P . . . O . . . . . D D
. . L . P . . . . . . . . . .
. D D D . . . . . . . P . L .

Время: Утро (Тик: 2)
Сущностей: 40
. . L . . . . . . . . . . . .
. . . . D M . . . . . . . . .
O P . D D . . . . O . . . . .
. . . D P . . L . . . . . . .
. . . . M . . . . O O . . . .
. . . . . . . . . . L . . . .
. . . . . . . . . . . . . . .
. . . L . . . D D D L . . . .
. O . . . . . D D . . . . . .
. . . . . . . . D . . . . . .
. . . . . . . . . . . . . D D
. . . P . . . . . . . . . D D
. . . . P . . O . . . . D D D
. . L . . . . . . . . . . . .
. D D D . . . . . . . . P L .

Время: Утро (Тик: 3)
Сущнос